In [5]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [6]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [7]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()


In [9]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=390)  

# Twelve Data API parameters
symbol = "XAU/USD"
interval = "1min"
outputsize = 5000  
time_series_data = []
indicator_data = []

# Increment through the time range in start_date and end_date by 5 days every 30 seconds
while start_date < end_date:
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=(start_date + timedelta(days=5)).strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)

    # Process and align data
    for index, row in time_series_df.iterrows():
        # Extract the datetime string
        datetime_str = row.name.strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append((
                symbol,
                datetime_str,
                row["open"],
                row["high"],
                row["low"],
                row["close"],
            ))
        if pd.notna(row["macd"]):  # Check if MACD is not NaN
            indicator_data.append(
                (
                    symbol,
                    datetime_str,
                    row["macd"],
                    row["macd_signal"],
                    row["macd_hist"],
                    row["ema"],
                    row["rsi"],
                )
            )

    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO stock_ticks (symbol, datetime, open, high, low, close)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)

    # Insert indicator data into MySQL
    sql_indicators = """
    INSERT INTO technical_indicators (symbol, bydatetime, macd, bysignal, byhistogram, ema, rsi)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_indicators, indicator_data)
    mydb.commit()

    # Increment the start_date
    start_date += timedelta(days=5)

    # Sleep for 30 seconds
    time.sleep(30)

DatabaseError: 1364 (HY000): Field 'volume' doesn't have a default value